In [ ]:
# pip install domolibrary

In [ ]:
import os

domo_username = os.environ.get("DOMO_USERNAME")
domo_instance = "domo-community"
domo_password = os.environ.get("DOJO_PASSWORD")

In [ ]:
import domolibrary.client.DomoAuth as dmda

auth = dmda.DomoFullAuth(
    domo_username=domo_username,
    domo_password=domo_password,
    domo_instance=domo_instance,
)

await auth.print_is_token()

🎉 full_auth token retrieved from domo-community ⚙️


True

In [ ]:
import domolibrary.classes.DomoDatacenter as dmdc

domo_datacenter = dmdc.DomoDatacenter(auth=auth)

dataflows_ls = await domo_datacenter.search_datacenter(
    auth=auth,
    entity_type="DATAFLOW",
    additional_filters_ls=[
        {
            "filterType": "term",
            "field": "data_flow_type",
            "value": "MAGIC",
            "name": "Magic ETL v2",
            "not": False,
        }
    ],
)

dataflows_ls[0]

{'entityType': 'dataflow',
 'databaseId': '185',
 'searchId': {'indexName': None,
  'databaseId': '185',
  'customer': 'mmmm-0012-0200',
  'entityType': 'dataflow'},
 'createDate': 1684450005000,
 'lastModified': 1686863874000,
 'lastIndexed': 1689352915038,
 'highlightedFields': {},
 'language': 'English',
 'requestAccess': False,
 'score': 64.42418,
 'name': 'DDX_Landing Page_DEV',
 'description': '',
 'ownedById': '1893952720',
 'ownedByName': 'Jae Wilson1',
 'tags': [],
 'lastRunDate': 1686863889000,
 'inputDatasets': [{'name': 'monit_user_accesslist',
   'id': '6ddbcb8d-0f38-48ad-bd73-f6b35c4b7daf'}],
 'outputDatasets': [{'name': 'DDX_Landing Page_DEV',
   'id': '15543b0f-dfe7-4a91-b7db-a86b9e8fec82'}],
 'runCount': 8,
 'successRate': 1.0,
 'dataFlowType': 'MAGIC',
 'status': 'SUCCESS',
 'deleted': False,
 'passwordProtected': False,
 'abandoned': False,
 'owners': [{'id': '1893952720',
   'type': 'USER',
   'displayName': 'Jae Wilson1'}],
 'ownersLocalized': {'localizedMessage': 

In [ ]:
dataflow_ids = [dataflow.get("databaseId") for dataflow in dataflows_ls]
dataflow_ids[0:5]

['185', '227', '340', '339', '309']

In [ ]:
import domolibrary.classes.DomoDataflow as dmdf
import asyncio

dataflow_definition_ls = await asyncio.gather(
    *[
        dmdf.DomoDataflow.get_from_id(dataflow_id=dataflow_id, auth=auth)
        for dataflow_id in dataflow_ids
    ]
)

domo_dataflow = dataflow_definition_ls[0]
domo_dataflow

DomoDataflow(id=185, name='DDX_Landing Page_DEV', owner=None, description=None, tags=None, actions=[DomoDataflow_Action(id='b9bb0f01-2517-4595-a077-c9778e5ffc48', type='LoadFromVault', name='monit_user_accesslist', datasource_id='6ddbcb8d-0f38-48ad-bd73-f6b35c4b7daf', sql=None, depends_on=None, parent_actions=None), DomoDataflow_Action(id='52f47029-06e0-4599-9ed8-21c4b9386c89', type='WindowAction', name='Rank & Window', datasource_id=None, sql=None, depends_on=['b9bb0f01-2517-4595-a077-c9778e5ffc48'], parent_actions=[DomoDataflow_Action(id='b9bb0f01-2517-4595-a077-c9778e5ffc48', type='LoadFromVault', name='monit_user_accesslist', datasource_id='6ddbcb8d-0f38-48ad-bd73-f6b35c4b7daf', sql=None, depends_on=None, parent_actions=None)]), DomoDataflow_Action(id='623883e9-bdc1-44cd-9fb4-6277d0e123de', type='ExpressionEvaluator', name='Add Formula', datasource_id=None, sql=None, depends_on=['52f47029-06e0-4599-9ed8-21c4b9386c89'], parent_actions=[DomoDataflow_Action(id='52f47029-06e0-4599-9ed8

In [ ]:
domo_dataflow = await dmdf.DomoDataflow.get_from_id(dataflow_id=9, auth=auth)
domo_dataflow

DomoDataflow(id=9, name='DOMO Covid Datasets', owner=None, description=None, tags=None, actions=[DomoDataflow_Action(id='0e192d7e-280b-4c4d-bde2-f0f3adb1e213', type='LoadFromVault', name='DOMO Covid Current Snapshot Tracker Data', datasource_id='b034f19e-16d5-4cb3-afac-b72c8e6a79fd', sql=None, depends_on=None, parent_actions=None), DomoDataflow_Action(id='96d04e4b-dc72-479c-b84b-ad0b74ada085', type='LoadFromVault', name='DOMO Covid Time Series Tracker Data', datasource_id='dcad2f50-e65e-4259-a9e8-214a3d1e18a7', sql=None, depends_on=None, parent_actions=None), DomoDataflow_Action(id='a1d07083-d1ca-41e9-8be2-f13596df03c3', type='ValueMapper', name='Value Mapper', datasource_id=None, sql=None, depends_on=['96d04e4b-dc72-479c-b84b-ad0b74ada085'], parent_actions=[DomoDataflow_Action(id='96d04e4b-dc72-479c-b84b-ad0b74ada085', type='LoadFromVault', name='DOMO Covid Time Series Tracker Data', datasource_id='dcad2f50-e65e-4259-a9e8-214a3d1e18a7', sql=None, depends_on=None, parent_actions=None)]

In [ ]:
import pandas as pd


async def generate_version_action_pdf(domo_dataflow):
    dataflow_versions = await domo_dataflow.get_versions()

    df = pd.DataFrame(
        [
            {
                "dataflow_id": dataflow_version.id,
                "dataflow_version": dataflow_version.version_id,
                **domo_action.__dict__,
            }
            for dataflow_version in dataflow_versions
            for domo_action in dataflow_version.actions
        ]
    )

    df.drop(columns=["parent_actions", "datasource_id", "sql"], inplace=True)
    df.rename(columns={"id": "tile_id", "type": "tile_type"}, inplace=True)

    return df


action_df = await generate_version_action_pdf(domo_dataflow)
action_df[0:5]

,dataflow_id,dataflow_version,tile_id,tile_type,name,depends_on
0,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None
1,9,89,96d04e4b-dc72-479c-b84b-ad0b74ada085,LoadFromVault,DOMO Covid Time Series Tracker Data,None
2,9,89,a1d07083-d1ca-41e9-8be2-f13596df03c3,ValueMapper,Value Mapper,[96d04e4b-dc72-479c-b84b-ad0b74ada085]
3,9,89,6e3ec9a4-470b-4fe4-8e8a-bd954b681df4,DateCalculator,Days Since...,[0e192d7e-280b-4c4d-bde2-f0f3adb1e213]
4,9,89,59e6226d-507f-4cff-86b1-f024499173aa,NumericCalculator,Resolved Cases,[6e3ec9a4-470b-4fe4-8e8a-bd954b681df4]


In [ ]:
async def generate_action_stats_df(domo_dataflow):
    await domo_dataflow.history.get_execution_history(maximum=10)

    df = pd.DataFrame(
        [
            {
                "dataflow_version": domo_history.dataflow_version,
                "dataflow_id": domo_history.dataflow_id,
                "dataflow_begin_time": domo_history.begin_time,
                **domo_action.__dict__,
            }
            for domo_history in domo_dataflow.history.execution_history
            for domo_action in domo_history.action_results
        ]
    )

    df.rename(columns={"id": "tile_id", "type": "tile_type"}, inplace=True)

    return df


stats_df = await generate_action_stats_df(domo_dataflow)
stats_df[0:5]

,dataflow_version,dataflow_id,dataflow_begin_time,tile_id,tile_type,name,is_success,rows_processed,begin_time,end_time,duration_in_sec
0,89,9,2023-12-08 22:46:34,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,DataHubManifestLoaderAction,None,True,315,2023-12-08 22:46:43.904,2023-12-08 22:46:45.072,1.168
1,89,9,2023-12-08 22:46:34,96d04e4b-dc72-479c-b84b-ad0b74ada085,DataHubManifestLoaderAction,None,True,389096,2023-12-08 22:46:43.878,2023-12-08 22:46:53.090,9.212
2,89,9,2023-12-08 22:46:34,a1d07083-d1ca-41e9-8be2-f13596df03c3,ValueMapper,None,True,389096,2023-12-08 22:46:44.873,2023-12-08 22:46:53.090,8.217
3,89,9,2023-12-08 22:46:34,6e3ec9a4-470b-4fe4-8e8a-bd954b681df4,DateCalculator,None,True,315,2023-12-08 22:46:44.964,2023-12-08 22:46:44.974,0.010
4,89,9,2023-12-08 22:46:34,59e6226d-507f-4cff-86b1-f024499173aa,NumericCalculator,None,True,315,2023-12-08 22:46:44.970,2023-12-08 22:46:44.974,0.004


In [ ]:
def combine_execution_plan_with_stats_df(action_df, stats_df):
    stats_df.drop(columns=["tile_type", "name"], inplace=True)

    df = pd.merge(
        action_df,
        stats_df,
        on=["dataflow_version", "dataflow_id", "tile_id"],
        how="outer",
    )

    return df


combine_df = combine_execution_plan_with_stats_df(action_df.copy(), stats_df.copy())

print(len(combine_df))
combine_df[:10]

931


,dataflow_id,dataflow_version,tile_id,tile_type,name,depends_on,dataflow_begin_time,is_success,rows_processed,begin_time,end_time,duration_in_sec
0,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None,2023-12-08 22:46:34,True,315.0,2023-12-08 22:46:43.904,2023-12-08 22:46:45.072,1.168
1,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None,2023-12-08 22:43:31,True,315.0,2023-12-08 22:43:41.863,2023-12-08 22:43:42.763,0.900
2,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None,2023-12-08 22:16:35,True,315.0,2023-12-08 22:16:45.120,2023-12-08 22:16:46.374,1.254
3,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None,2023-12-08 22:13:28,True,315.0,2023-12-08 22:13:37.857,2023-12-08 22:13:39.008,1.151
4,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None,2023-12-08 21:46:03,True,315.0,2023-12-08 21:46:13.758,2023-12-08 21:46:14.765,1.007
5,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None,2023-12-08 21:43:33,True,315.0,2023-12-08 21:43:42.677,2023-12-08 21:43:43.873,1.196
6,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None,2023-12-08 21:16:25,True,315.0,2023-12-08 21:16:34.167,2023-12-08 21:16:35.477,1.310
7,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None,2023-12-08 21:13:29,True,315.0,2023-12-08 21:13:39.949,2023-12-08 21:13:41.180,1.231
8,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None,2023-12-08 20:48:59,True,315.0,2023-12-08 20:49:08.108,2023-12-08 20:49:09.089,0.981
9,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,None,2023-12-08 20:48:12,True,315.0,2023-12-08 20:48:21.103,2023-12-08 20:48:22.189,1.086


In [ ]:
def explode_parents(df):
    """
    1. generates one row for each parent in "depends_on"
    2. joins parents stats to df
    3. stores dataflow_begin_time if there is no parent
    """

    df["dependencies"] = df["depends_on"]
    explode_df = df.explode("depends_on")

    parents_df = df[
        ["dataflow_version", "dataflow_id", "tile_id", "tile_type", "name", "end_time", "rows_processed"]
    ].rename(
        columns={
            "tile_id": "parent_tile_id",
            "tile_type": "parent_tile_type",
            "name": "parent_tile_name",
            "end_time": "parent_end_time",
            "rows_processed" : "parent_rows_processed"
        }
    )

    explode_df = pd.merge(
        explode_df,
        parents_df,
        left_on=["dataflow_version", "dataflow_id", "depends_on"],
        right_on=["dataflow_version", "dataflow_id", "parent_tile_id"],
        how="left",
    )

    explode_df["parent_end_time"] = explode_df[["begin_time", "parent_end_time"]].max(
        axis=1
    )

    explode_df["actual_duration_in_sec"] = (
        explode_df["end_time"] - explode_df["parent_end_time"]
    ).dt.total_seconds()

    explode_df = explode_df[
        [
            "dataflow_id",
            "dataflow_version",
            "tile_id",
            "tile_type",
            "name",
            "rows_processed",
            "begin_time",
            "end_time",
            "duration_in_sec",
            "parent_end_time",
            "actual_duration_in_sec",
            "dependencies",
            "parent_tile_id",
            "parent_tile_type",
            "parent_tile_name",
            "parent_rows_processed",
        ]
    ]

    explode_df["parent_tile_id"].fillna("start", inplace=True)

    return explode_df


explode_df = explode_parents(combine_df.copy())
explode_df[0:10]

,dataflow_id,dataflow_version,tile_id,tile_type,name,rows_processed,begin_time,end_time,duration_in_sec,parent_end_time,actual_duration_in_sec,dependencies,parent_tile_id,parent_tile_type,parent_tile_name,parent_rows_processed
0,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,315.0,2023-12-08 22:46:43.904,2023-12-08 22:46:45.072,1.168,2023-12-08 22:46:43.904000000,1.168,None,start,NaN,NaN,NaN
1,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,315.0,2023-12-08 22:43:41.863,2023-12-08 22:43:42.763,0.900,2023-12-08 22:43:41.863000064,0.900,None,start,NaN,NaN,NaN
2,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,315.0,2023-12-08 22:16:45.120,2023-12-08 22:16:46.374,1.254,2023-12-08 22:16:45.120000000,1.254,None,start,NaN,NaN,NaN
3,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,315.0,2023-12-08 22:13:37.857,2023-12-08 22:13:39.008,1.151,2023-12-08 22:13:37.856999936,1.151,None,start,NaN,NaN,NaN
4,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,315.0,2023-12-08 21:46:13.758,2023-12-08 21:46:14.765,1.007,2023-12-08 21:46:13.758000128,1.007,None,start,NaN,NaN,NaN
5,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,315.0,2023-12-08 21:43:42.677,2023-12-08 21:43:43.873,1.196,2023-12-08 21:43:42.676999936,1.196,None,start,NaN,NaN,NaN
6,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,315.0,2023-12-08 21:16:34.167,2023-12-08 21:16:35.477,1.310,2023-12-08 21:16:34.167000064,1.310,None,start,NaN,NaN,NaN
7,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,315.0,2023-12-08 21:13:39.949,2023-12-08 21:13:41.180,1.231,2023-12-08 21:13:39.948999936,1.231,None,start,NaN,NaN,NaN
8,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,315.0,2023-12-08 20:49:08.108,2023-12-08 20:49:09.089,0.981,2023-12-08 20:49:08.108000000,0.981,None,start,NaN,NaN,NaN
9,9,89,0e192d7e-280b-4c4d-bde2-f0f3adb1e213,LoadFromVault,DOMO Covid Current Snapshot Tracker Data,315.0,2023-12-08 20:48:21.103,2023-12-08 20:48:22.189,1.086,2023-12-08 20:48:21.103000064,1.086,None,start,NaN,NaN,NaN


In [ ]:
def calculate_average(df):
    """calculates average execution time for each tile, differentiating by reported duration_in_sec vs actual_duration which assumes an upstream parent"""

    return df.groupby(
        ["dataflow_id", "dataflow_version", "tile_id", "parent_tile_id"]
    ).agg(
        {
            "name": "first",
            "tile_type": "first",
            "parent_tile_name": "first",
            "parent_tile_type": "first",
            "duration_in_sec": "mean",
            "actual_duration_in_sec": "mean",
            "rows_processed": "mean",
            "parent_rows_processed": "mean"
        }
    )


calculate_average(explode_df.copy())

name  \
dataflow_id dataflow_version tile_id                              parent_tile_id                                             
9           20               008ac1ce-4143-420f-93a7-9536ea986fc5 e667577a-8c73-4677-8022-8bf3f3e7a72e       Rank & Window   
                             02a1f8dd-2b0a-4193-9c54-8d4fc19261cc 250bf67d-8882-4078-a7d4-aa6dda1a1635      Snapshot Age 1   
                             0362dd20-2039-403d-a249-94e2ce716aed 18626a7b-c1a7-4fb4-bcc0-7e50ebd7e763               Today   
                             06f8d60b-8b84-474d-91de-abcaefe10abf 0e783798-e39f-49f2-ac42-70cd82ffd611  DIM_Country (Join)   
                             08327437-2a70-49bb-9676-8e63f540eb84 5da719b4-d4c6-4dfc-a928-38312de82df8    Null Recovered 2   
...                                                                                                                    ...   
            89               d652deb2-cd3e-462d-9627-2f72425e05cf 08327437-2a70-49bb-9676-8e63f540eb84  Null New Recovered   
                             dac9af4e-a203-47f5-9493-ec46d834a51f cebf2fb3-75e3-48ee-bfe5-2af9927cb581    Resolved Cases 2   
                             dc5a2d0c-3bdf-4e94-a07a-7ff4709acc1f 1d3e790a-dabf-45ba-849e-74996d7a03c9      Active Cases 1   
                             eeecc67b-34c4-4929-a4e3-34058aba039a 43029ef0-9519-49c6-80c2-2f885e662221             Ranks 1   
                             fe46f53d-d10e-4e1e-b341-4d18f565e83b d652deb2-cd3e-462d-9627-2f72425e05cf       Null or 0 Lat   

                                                                                                                tile_type  \
dataflow_id dataflow_version tile_id                              parent_tile_id                                            
9           20               008ac1ce-4143-420f-93a7-9536ea986fc5 e667577a-8c73-4677-8022-8bf3f3e7a72e       WindowAction   
                             02a1f8dd-2b0a-4193-9c54-8d4fc19261cc 250bf67d-8882-4078-a7d4-aa6dda1a1635     DateCalculator   
                             0362dd20-2039-403d-a249-94e2ce716aed 18626a7b-c1a7-4fb4-bcc0-7e50ebd7e763           Constant   
                             06f8d60b-8b84-474d-91de-abcaefe10abf 0e783798-e39f-49f2-ac42-70cd82ffd611       SelectValues   
                             08327437-2a70-49bb-9676-8e63f540eb84 5da719b4-d4c6-4dfc-a928-38312de82df8        ValueMapper   
...                                                                                                                   ...   
            89               d652deb2-cd3e-462d-9627-2f72425e05cf 08327437-2a70-49bb-9676-8e63f540eb84        ValueMapper   
                             dac9af4e-a203-47f5-9493-ec46d834a51f cebf2fb3-75e3-48ee-bfe5-2af9927cb581  NumericCalculator   
                             dc5a2d0c-3bdf-4e94-a07a-7ff4709acc1f 1d3e790a-dabf-45ba-849e-74996d7a03c9  NumericCalculator   
                             eeecc67b-34c4-4929-a4e3-34058aba039a 43029ef0-9519-49c6-80c2-2f885e662221       SelectValues   
                             fe46f53d-d10e-4e1e-b341-4d18f565e83b d652deb2-cd3e-462d-9627-2f72425e05cf        ValueMapper   

                                                                                                                   parent_tile_name  \
dataflow_id dataflow_version tile_id                              parent_tile_id                                                      
9           20               008ac1ce-4143-420f-93a7-9536ea986fc5 e667577a-8c73-4677-8022-8bf3f3e7a72e  Append Unused DIM Countries   
                             02a1f8dd-2b0a-4193-9c54-8d4fc19261cc 250bf67d-8882-4078-a7d4-aa6dda1a1635               Active Cases 2   
                             0362dd20-2039-403d-a249-94e2ce716aed 18626a7b-c1a7-4fb4-bcc0-7e50ebd7e763       Countries Still Listed   
                             06f8d60b-8b84-474d-91de-abcaefe10abf 0e783798-e39f-49f2-ac42-70cd82ffd611            Append New Values   
                             08327437-2a70-

In [ ]:
def get_action_list(domo_dataflow: dmdf.DomoDataflow):
    return [
        {
            "dataflow_id": domo_dataflow.id,
            "datafow_name": domo_dataflow.name,
            "action_type": domo_action.type,
        }
        for domo_action in domo_dataflow.actions
    ]


get_action_list(domo_dataflow=dataflow_definition_ls[0])

[{'dataflow_id': 185,
  'datafow_name': 'DDX_Landing Page_DEV',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 185,
  'datafow_name': 'DDX_Landing Page_DEV',
  'action_type': 'WindowAction'},
 {'dataflow_id': 185,
  'datafow_name': 'DDX_Landing Page_DEV',
  'action_type': 'ExpressionEvaluator'},
 {'dataflow_id': 185,
  'datafow_name': 'DDX_Landing Page_DEV',
  'action_type': 'Metadata'},
 {'dataflow_id': 185,
  'datafow_name': 'DDX_Landing Page_DEV',
  'action_type': 'SelectValues'},
 {'dataflow_id': 185,
  'datafow_name': 'DDX_Landing Page_DEV',
  'action_type': 'ExpressionEvaluator'},
 {'dataflow_id': 185,
  'datafow_name': 'DDX_Landing Page_DEV',
  'action_type': 'PublishToVault'}]

In [ ]:
action_ls = [get_action_list(dd_obj) for dd_obj in dataflow_definition_ls]

flat_action_ls = [action for a_list in action_ls for action in a_list]
flat_action_ls[0:5]

[{'dataflow_id': 185,
  'datafow_name': 'DDX_Landing Page_DEV',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 185,
  'datafow_name': 'DDX_Landing Page_DEV',
  'action_type': 'WindowAction'},
 {'dataflow_id': 185,
  'datafow_name': 'DDX_Landing Page_DEV',
  'action_type': 'ExpressionEvaluator'},
 {'dataflow_id': 185,
  'datafow_name': 'DDX_Landing Page_DEV',
  'action_type': 'Metadata'},
 {'dataflow_id': 185,
  'datafow_name': 'DDX_Landing Page_DEV',
  'action_type': 'SelectValues'}]

In [ ]:
import pandas as pd

df = pd.DataFrame(flat_action_ls)
df[0:5]

,dataflow_id,datafow_name,action_type
0,185,DDX_Landing Page_DEV,LoadFromVault
1,185,DDX_Landing Page_DEV,WindowAction
2,185,DDX_Landing Page_DEV,ExpressionEvaluator
3,185,DDX_Landing Page_DEV,Metadata
4,185,DDX_Landing Page_DEV,SelectValues


In [ ]:
df.to_csv("marks_csv.csv")